Import the required packages

In [30]:
import torch
import os
from torch import nn
import torchvision  
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [32]:
batch_size=256
epochs=30

Data import

In [33]:
transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(0.5),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
# 下载FashionMNIST数据集  
fmnist_train = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform)
fmnist_text = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform) 
# 定义数据加载器
train_loader = torch.utils.data.DataLoader(
    fmnist_train, batch_size=batch_size, shuffle=True)
text_loader = torch.utils.data.DataLoader(
    fmnist_text, batch_size=batch_size, shuffle=False)

Define the network

In [39]:
class Netcnn(nn.Module):
    def __init__(self):
        super(Netcnn, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.3)
        )
        
        
        self.fc = nn.Sequential(
            nn.Linear(64*4*4, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 64*4*4)  # Flatten the tensor
        x = self.fc(x)
        return x
    
class Netmlp(nn.Module):
    def __init__(self):
        super(Netmlp,self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 256),  
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.model(x)


In [35]:
netcnn=Netcnn().to(device)
netmlp=Netmlp().to(device)
loss = nn.CrossEntropyLoss()
optimizer1 = torch.optim.Adam(netcnn.parameters(), lr=1e-4)
optimizer2 = torch.optim.Adam(netmlp.parameters(), lr=0.001,weight_decay=1e-4)

Define the function

In [36]:
def train(epoch,net,optimizer): 
    net.train()
    train_loss=0
    for batch, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_loss += l.item()*X.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
        
def validate(epoch,net,optimizer):
    net.eval()
    val_loss=0
    gt_labels=[]
    pred_labels=[]
    with torch.no_grad():
        for batch, (X, y) in enumerate(text_loader):
            X, y = X.to(device), y.to(device)   
            y_hat=net(X)
            preds=torch.argmax(y_hat,1)
            gt_labels.append(y.cpu().data.numpy())
            pred_labels.append(preds.cpu().data.numpy())
            l = loss(y_hat, y)
            val_loss+=l.item()*X.size(0)
    val_loss=val_loss/len(text_loader.dataset)
    gt_labels, pred_labels = np.concatenate(gt_labels), np.concatenate(pred_labels)
    acc = np.sum(gt_labels==pred_labels)/len(pred_labels)
    print('Epoch: {} \tValidation Loss: {:.6f}, Accuracy: {:6f}'.format(epoch, val_loss, acc))
    

Train CNN Model

In [37]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=10, gamma=0.5)
for epoch in range(1, epochs+1):
    train(epoch, netcnn, optimizer1)
    validate(epoch, netcnn, optimizer1)
    scheduler.step()


Epoch: 1 	Training Loss: 0.992943
Epoch: 1 	Validation Loss: 0.637074, Accuracy: 0.760900
Epoch: 2 	Training Loss: 0.601747
Epoch: 2 	Validation Loss: 0.550952, Accuracy: 0.795600
Epoch: 3 	Training Loss: 0.540766
Epoch: 3 	Validation Loss: 0.502564, Accuracy: 0.815400
Epoch: 4 	Training Loss: 0.498180
Epoch: 4 	Validation Loss: 0.463746, Accuracy: 0.834400
Epoch: 5 	Training Loss: 0.465276
Epoch: 5 	Validation Loss: 0.444925, Accuracy: 0.840700
Epoch: 6 	Training Loss: 0.439689
Epoch: 6 	Validation Loss: 0.418746, Accuracy: 0.852100
Epoch: 7 	Training Loss: 0.420472
Epoch: 7 	Validation Loss: 0.399510, Accuracy: 0.858600
Epoch: 8 	Training Loss: 0.404426
Epoch: 8 	Validation Loss: 0.387895, Accuracy: 0.863900
Epoch: 9 	Training Loss: 0.391795
Epoch: 9 	Validation Loss: 0.380782, Accuracy: 0.864100
Epoch: 10 	Training Loss: 0.380440
Epoch: 10 	Validation Loss: 0.372618, Accuracy: 0.867600
Epoch: 11 	Training Loss: 0.368520
Epoch: 11 	Validation Loss: 0.362353, Accuracy: 0.869300
Epoch:

Train MLP Model

In [40]:
for epoch in range(1, epochs+1):
    train(epoch,netmlp,optimizer2)
    validate(epoch,netmlp,optimizer2)

Epoch: 1 	Training Loss: 0.159177
Epoch: 1 	Validation Loss: 0.366035, Accuracy: 0.888000
Epoch: 2 	Training Loss: 0.159315
Epoch: 2 	Validation Loss: 0.344176, Accuracy: 0.891100
Epoch: 3 	Training Loss: 0.151427
Epoch: 3 	Validation Loss: 0.353249, Accuracy: 0.892900
Epoch: 4 	Training Loss: 0.148462
Epoch: 4 	Validation Loss: 0.381586, Accuracy: 0.890800
Epoch: 5 	Training Loss: 0.144204
Epoch: 5 	Validation Loss: 0.379413, Accuracy: 0.890200
Epoch: 6 	Training Loss: 0.144399
Epoch: 6 	Validation Loss: 0.383760, Accuracy: 0.887100
Epoch: 7 	Training Loss: 0.141030
Epoch: 7 	Validation Loss: 0.389977, Accuracy: 0.886000
Epoch: 8 	Training Loss: 0.138712
Epoch: 8 	Validation Loss: 0.366543, Accuracy: 0.891300
Epoch: 9 	Training Loss: 0.133808
Epoch: 9 	Validation Loss: 0.395987, Accuracy: 0.894700
Epoch: 10 	Training Loss: 0.132046
Epoch: 10 	Validation Loss: 0.371676, Accuracy: 0.893000
Epoch: 11 	Training Loss: 0.126154
Epoch: 11 	Validation Loss: 0.391541, Accuracy: 0.887700
Epoch: